# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [61]:
import numpy as np
import os
from PIL import Image
import datetime
import os
from zipfile import ZipFile
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

We set the random seed so that the results don't vary drastically.

In [62]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

##### Extract the Data

In [ ]:
# extract_location = '/content/gdrive/MyDrive/Colab Notebooks/gesture recognition/'
# zip_file = '/content/gdrive/MyDrive/Colab Notebooks/gesture recognition/Project_data.zip'
# with ZipFile(zip_file, 'r') as zObject:
#   zObject.extractall(
#         path=extract_location)

##### Check GPU

In [63]:
device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
else:
    print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


2023-02-08 18:01:23.966455: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /device:GPU:0 with 14802 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:1c:00.0, compute capability: 7.5


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [64]:
train_doc = np.random.permutation(open('datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('datasets/Project_data/val.csv').readlines())

##### Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [65]:
def getBatchData(source_path, t, batch, batch_size, img_tensor):
    [x,y,z] = [len(img_tensor[0]),img_tensor[1], img_tensor[2]]
    img_idx = img_tensor[0]
    batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
    batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
    for folder in range(batch_size): # iterate over the batch_size
        imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
        for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
            image = Image.open(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item])#.astype(np.float32)
            width, height = image.size
            # crop the images and resize them. Note that the images are of 2 different shape 
            #and the conv3D will throw error if the inputs in a batch have different shapes
            image = image.resize((128,128))

            batch_data[folder,idx,:,:,0] = np.array(image)[:, :, 0]/255 #normalise and feed in the image
            batch_data[folder,idx,:,:,1] = np.array(image)[:, :, 1]/255 #normalise and feed in the image
            batch_data[folder,idx,:,:,2] = np.array(image)[:, :, 2]/255 #normalise and feed in the image

        batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
    return batch_data, batch_labels

In [66]:
def generator(source_path, folder_list, batch_size, img_tensor):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(folder_list)/batch_size)
        for batch in range(num_batches): # we iterate over the number of batches
            yield getBatchData(source_path, t, batch, batch_size, img_tensor)
        
        # write the code for the remaining data points which are left after full batches
        # checking if any remaining batches are there or not
        if len(folder_list)%batch_size != 0:
            batch_size = len(folder_list)%batch_size
            yield getBatchData(source_path, t, batch, batch_size, img_tensor)

In [67]:
def ImageToTensor(n_frames):
    img_idx = np.round(np.linspace(0, 29, n_frames)).astype(int)
    return [img_idx, 128, 128, 3]

# define image tensor size
img_tensor = ImageToTensor(15)
print ('Image Tensor:', img_tensor)

# img_tensor = [array([ 0,  2,  4,  6,  8, 10, 12, 15, 17, 19, 21, 23, 25, 27, 29]), 128, 128, 3]


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [68]:
curr_dt_time = datetime.datetime.now()
train_path = 'datasets/Project_data/train'
val_path = 'datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)

# training sequences = 663
# validation sequences = 100


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [69]:
batch_size = 32
train_generator = generator(train_path, train_doc, batch_size, img_tensor)
val_generator = generator(val_path, val_doc, batch_size, img_tensor)
batch_data, batch_labels = next(train_generator)
signs = ['Backward','Forward' ,'Stop' , 'Volume Down' ,'Volume Up']
print(signs[batch_labels[4].argmax()])
for i in range(15):
  plt.figure()
  plt.imshow(batch_data[4][i])

## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [70]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout, LSTM, Input
from keras.layers.convolutional import Conv3D, MaxPooling3D, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from tensorflow.keras.applications import ResNet50, VGG16, MobileNet

#write your model here

resnet = ResNet50(include_top=False,weights='imagenet')
n_frames = 15
num_epochs = 10
input_shape = (n_frames, 128,128,3)
for layer in resnet.layers:
  layer.trainable = False

def build_model():
  model = Sequential([TimeDistributed(resnet, input_shape=input_shape)], name="resnet_lstm")

  model.add(TimeDistributed(BatchNormalization()))
  model.add(TimeDistributed(MaxPooling2D((2,2))))
  model.add(TimeDistributed(Flatten()))

  model.add(LSTM(256))
  model.add(Dropout(0.25))
  
  model.add(Dense(256,activation='relu'))
  model.add(Dropout(0.25))
  
  model.add(Dense(5, activation='softmax'))
  
  return model

In [71]:
model = build_model()
optimiser = 'adam'
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

Model: "resnet_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_38 (TimeDi  (None, 15, 4, 4, 2048)   23587712  
 stributed)                                                      
                                                                 
 time_distributed_39 (TimeDi  (None, 15, 4, 4, 2048)   8192      
 stributed)                                                      
                                                                 
 time_distributed_40 (TimeDi  (None, 15, 2, 2, 2048)   0         
 stributed)                                                      
                                                                 
 time_distributed_41 (TimeDi  (None, 15, 8192)         0         
 stributed)                                                      
                                                                 
 lstm_11 (LSTM)              (None, 256)               

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

##### Checkpoint

In [72]:
model_name = 'models/gesture recognition/Checkpoints/model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [73]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [74]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=15, verbose=1, 
                    callbacks=callbacks_list,validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  datasets/Project_data/train ; batch size = 32
Epoch 1/15


2023-02-08 18:02:11.846871: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.89GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2023-02-08 18:02:12.034532: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.89GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


21/21 [==============================] - ETA: 0s - loss: 1.0142 - categorical_accuracy: 0.5867Source path =  datasets/Project_data/val ; batch size = 32

Epoch 00001: saving model to models/gesture recognition/Checkpoints/model_init_2023-02-0818_01_34.640316/model-00001-1.01420-0.58673-1.85184-0.22000.h5
21/21 [==============================] - 53s 2s/step - loss: 1.0142 - categorical_accuracy: 0.5867 - val_loss: 1.8518 - val_categorical_accuracy: 0.2200 - lr: 0.0010
Epoch 2/15
21/21 [==============================] - ETA: 0s - loss: 0.6104 - categorical_accuracy: 0.7805
Epoch 00002: saving model to models/gesture recognition/Checkpoints/model_init_2023-02-0818_01_34.640316/model-00002-0.61044-0.78054-3.18976-0.25000.h5
21/21 [==============================] - 27s 1s/step - loss: 0.6104 - categorical_accuracy: 0.7805 - val_loss: 3.1898 - val_categorical_accuracy: 0.2500 - lr: 0.0010
Epoch 3/15
21/21 [==============================] - ETA: 0s - loss: 0.6999 - categorical_accuracy: 0.751

KeyboardInterrupt: 